In [4]:
import numpy as np
import pandas as pd
import pickle

In [10]:
with open("test/track_infos_0_166899.pickle", "rb") as f:
    data = pickle.load(f)

data

{'0UaMYEvWZi0ZqiDOoHU3YI': {'artist_name': 'Missy Elliott',
  'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
  'audio_feats': {'danceability': 0.904,
   'energy': 0.813,
   'key': 4,
   'loudness': -7.105,
   'mode': 0,
   'speechiness': 0.121,
   'acousticness': 0.0311,
   'instrumentalness': 0.00697,
   'liveness': 0.0471,
   'valence': 0.81,
   'tempo': 125.461,
   'duration_ms': 226864,
   'time_signature': 4}},
 '6I9VzXrHxO9rA9A5euc8Ak': {'artist_name': 'Britney Spears',
  'track_name': 'Toxic',
  'audio_feats': {'danceability': 0.774,
   'energy': 0.838,
   'key': 5,
   'loudness': -3.914,
   'mode': 0,
   'speechiness': 0.114,
   'acousticness': 0.0249,
   'instrumentalness': 0.025,
   'liveness': 0.242,
   'valence': 0.924,
   'tempo': 143.04,
   'duration_ms': 198800,
   'time_signature': 4}},
 '0WqIKmW4BTrj3eJFmnCKMv': {'artist_name': 'Beyoncé',
  'track_name': 'Crazy In Love',
  'audio_feats': {'danceability': 0.664,
   'energy': 0.758,
   'key': 2,
   'loudness

In [13]:
pd.json_normalize(data["0UaMYEvWZi0ZqiDOoHU3YI"])

,artist_name,track_name,audio_feats.danceability,audio_feats.energy,audio_feats.key,audio_feats.loudness,audio_feats.mode,audio_feats.speechiness,audio_feats.acousticness,audio_feats.instrumentalness,audio_feats.liveness,audio_feats.valence,audio_feats.tempo,audio_feats.duration_ms,audio_feats.time_signature
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,125.461,226864,4
